In [ ]:
import numpy as np
import pandas
import pandas as pd

In [ ]:
import rbot
from rbot import init_log;
from rbot import Market
from rbot import Session
from rbot import BaseAgent
from rbot import NOW
from rbot import DAYS




In [ ]:
init_log()
Market.dummy_mode()
Market.open("FTX", "BTC-PERP")


In [ ]:

class Agent(BaseAgent):   
    def clock_interval(self):
        return 60    # Sec
    
    def on_tick(self, time, price, side, size):
        print("tick", time, price, side, size)
        print(self.session.current_timestamp)

In [ ]:
session = Session(market=Market.get("FTX", "BTC-PERP"), agent=Agent())

In [ ]:
session.run(NOW()-1_000_000 * 3600, NOW())

In [ ]:
session.center_price

In [ ]:
session.current_timestamp

In [ ]:
session.run()

In [ ]:
session.current_timestamp

In [ ]:
session.long_order_size

In [ ]:
session.make_order(1, rbot.OrderSide.Buy, 10.0, 10.0, 10, "")

In [ ]:
session.long_orders

In [ ]:
session.long_order_size

In [ ]:
class Market:
    MARKET = {}
    DUMMY_MODE = True

    @classmethod
    def dummy_mode(cls, dummy=True):
        cls.DUMMY_MODE = dummy

    @classmethod
    def open(cls, exchange: str, market):
        exchange = exchange.upper()
        if exchange == "FTX":
            m = rbot.FtxMarket(market, cls.DUMMY_MODE)
            cls.MARKET[exchange.upper() + "/" + market.upper()] = m
            return m
        

    @classmethod
    def download(cls, ndays):
        for m in cls.MARKET:
            cls.MARKET[m].download(ndays)
    
    @classmethod
    def get(cls, exchange, market):
        return cls.MARKET[exchange.upper() + "/" + market.upper()]
            

In [ ]:
rbot.init_log()
Market.dummy_mode()
Market.open("FTX", "BTC-PERP")
Market.download(1)
Market.get("FTX", "BTC-PERP")

In [ ]:
ftx = Market.get("FTX", "BTC-PERP")

In [ ]:
ftx.info()

In [ ]:
class Session:
    def __init__(self, market, agent):
        self.session = rbot._DummySession()
        self.market = market
        self.agent = agent

        if hasattr(self.agent, 'clock_interval'):
            self.clock_interval = self.agent.clock_interval()
        else:
            print("Agent class has no clock_interval() sttings, use default 60[sec]");            
            self.clock_interval = 60

        
    def __getattr__(self, func):
        return getattr(self.session, func)

                                       
    def want_on_tick(self):
        return hasattr(self.agent, 'on_tick')

    def want_on_clock(self):
        return hasattr(self.agent, 'on_clock')
    
    def want_on_update(self):
        return hasattr(self.agent, 'on_update')
    
    def make_orders(self, orders):
        for order in orders:
            self.make_order(order)
    
    def make_order(self, order):
        self.session.make_order(order)
    
    def on_tick(self, time, price, side, size):
        return self.agent.on_tick(self, time, price, side, size)
    
    def on_clock(self):
        return self.agent.on_clock(self, time)
        
    def on_update(self, time, order_result):
        return self.agent.on_update(self, time, order_result)
        
    def ohlcvv(self, time_window, num_of_bars, exchange_name=None, market_name=None):
        if market_name == None:
            market = self.market
        else:
            market = Market.get(exchange_name, market_name)

        now = self.session.current_timestamp
        return market.ohlcvv(now - time_window * num_of_bars * 1_000_000, now, time_window)

        

In [ ]:
s = Session(ftx, Agent())

In [ ]:
s.want_on_tick()

In [ ]:
s.on_tick(1, 1, "buy", 10.0)

In [ ]:
ohlcv = s.ohlcvv(60, 3)

In [ ]:
ohlcv2 = Market.get("FTX", "BTC-PERP").ohlcvv(rbot.NOW()- 1_000_000*60*10, rbot.NOW(), 10)

In [ ]:
ohlcv2 = Market.get("FTX", "BTC-PERP").ohlcvv(- 1_000_000*60*10, 1, 10)


In [ ]:
ohlcv2

In [ ]:
s = rbot.Session()

In [ ]:
s

In [ ]:
s2 = rbot._DummySession()

In [ ]:
s2.get_center_price()

In [ ]:
df2 = pd.read_csv('ftx.csv', names=("timestamp", 'price', 'order_side', 'liquid', 'id'), i)

In [ ]:
df2

In [ ]:
df2['diff'] = df2['timestamp'].diff()

In [ ]:
df2['diff']

In [ ]:
df2[df2['diff'] < -1]